<a href="https://colab.research.google.com/github/agemagician/CodeTrans/blob/main/prediction/multitask/pre-training/function%20documentation%20generation/python/large_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**<h3>Predict the documentation for python code using codeTrans multitask training model</h3>**
<h4>You can make free prediction online through this 
<a href="https://huggingface.co/SEBIS/code_trans_t5_large_code_documentation_generation_python_multitask">Link</a></h4> (When using the prediction online, you need to parse and tokenize the code first.)

**1. Load necessry libraries including huggingface transformers**

In [1]:
!pip install -q transformers sentencepiece

     |████████████████████████████████| 1.8MB 10.9MB/s 
     |████████████████████████████████| 1.2MB 42.4MB/s 
     |████████████████████████████████| 890kB 23.9MB/s 
     |████████████████████████████████| 3.2MB 39.5MB/s 


In [2]:
from transformers import AutoTokenizer, AutoModelWithLMHead, SummarizationPipeline

**2. Load the token classification pipeline and load it into the GPU if avilabile**

In [3]:
pipeline = SummarizationPipeline(
    model=AutoModelWithLMHead.from_pretrained("SEBIS/code_trans_t5_large_code_documentation_generation_python_multitask"),
    tokenizer=AutoTokenizer.from_pretrained("SEBIS/code_trans_t5_large_code_documentation_generation_python_multitask", skip_special_tokens=True),
    device=0
)

/usr/local/lib/python3.6/dist-packages/transformers/models/auto/modeling_auto.py:970: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


**3 Give the code for summarization, parse and tokenize it**

In [4]:
code = "def e(message, exit_code=None):\n   print_log(message, YELLOW, BOLD)\n    if exit_code is not None:\n        sys.exit(exit_code)" #@param {type:"raw"}


In [5]:
!pip install tree_sitter
!git clone https://github.com/tree-sitter/tree-sitter-python

     |████████████████████████████████| 112kB 10.5MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for tree-sitter: filename=tree_sitter-0.2.2-cp36-cp36m-linux_x86_64.whl size=297839 sha256=9e150128171fb10e71c61b0d8aaa912607373f811c3c43cdb9232d9aeead84d1
  Stored in directory: /root/.cache/pip/wheels/2b/c0/0f/32967a2a849330fe51b835c77efae3f83b903a40f5132102d3
Successfully built tree-sitter
Cloning into 'tree-sitter-python'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 2301 (delta 21), reused 32 (delta 16), pack-reused 2256
Receiving objects: 100% (2301/2301), 18.13 MiB | 10.42 MiB/s, done.
Resolving deltas: 100% (1450/1450), done.


In [6]:
from tree_sitter import Language, Parser

Language.build_library(
  'build/my-languages.so',
  ['tree-sitter-python']
)

PYTHON_LANGUAGE = Language('build/my-languages.so', 'python')
parser = Parser()
parser.set_language(PYTHON_LANGUAGE)

In [7]:
def get_string_from_code(node, lines):
  line_start = node.start_point[0]
  line_end = node.end_point[0]
  char_start = node.start_point[1]
  char_end = node.end_point[1]
  if line_start != line_end:
    code_list.append(' '.join([lines[line_start][char_start:]] + lines[line_start+1:line_end] + [lines[line_end][:char_end]]))
  else:
    code_list.append(lines[line_start][char_start:char_end])

def my_traverse(node, code_list):
  lines = code.split('\n')
  if node.child_count == 0:
    get_string_from_code(node, lines)
  elif node.type == 'string':
    get_string_from_code(node, lines)
  else:
    for n in node.children:
      my_traverse(n, code_list)
 
  return ' '.join(code_list)

In [8]:
tree = parser.parse(bytes(code, "utf8"))
code_list=[]
tokenized_code = my_traverse(tree.root_node, code_list)
print("Output after tokenization: " + tokenized_code)

Output after tokenization: def e ( message , exit_code = None ) : print_log ( message , YELLOW , BOLD ) if exit_code is not None : sys . exit ( exit_code )


**4. Make Prediction**

In [9]:
pipeline([tokenized_code])

[{'summary_text': 'Print an error message and exit with the specified exit code .'}]